# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings; warnings.simplefilter('ignore')

import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

clean_weather_data = pd.read_csv("../output_data/clean_weather_data.csv")
clean_weather_data


,Unnamed: 0,City,Country,ID,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Hobart,AU,2163355,0,66,-42.88,147.33,50.00,12.75
1,1,Pisco,PE,3932145,2,93,-13.70,-76.22,59.00,4.70
2,2,Presidente dutra,BR,3391220,25,72,-5.29,-44.49,73.00,0.20
3,3,Hobart,AU,2163355,0,66,-42.88,147.33,50.00,12.75
4,4,Saint-philippe,RE,935215,50,77,-21.36,55.77,68.00,5.82
...,...,...,...,...,...,...,...,...,...,...
1170,1170,Grand gaube,MU,934479,45,77,-20.01,57.66,73.00,5.01
1171,1171,Berea,US,5147097,7,85,41.37,-81.85,73.99,9.35
1172,1172,Ushuaia,AR,3833367,20,59,-54.80,-68.30,35.60,20.80
1173,1173,Bluff,NZ,2206939,100,93,-46.60,168.33,49.80,19.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)
weights = clean_weather_data["Humidity"]
fig = gmaps.figure(map_type="SATELLITE")
heatmap_layer = gmaps.heatmap_layer(clean_weather_data[["Latitude", "Longitude"]],weights=weights)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
narrow_data = clean_weather_data.loc[(clean_weather_data["Max Temp"] < 80) &
                                     (clean_weather_data["Max Temp"] > 70) &
                                     (clean_weather_data["Wind Speed"] < 10) & 
                                     (clean_weather_data["Cloudiness"] == 0)]
narrow_Data = narrow_data.dropna()
narrow_data

,Unnamed: 0,City,Country,ID,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed
86,86,Zhangye,CN,1785036,0,26,38.93,100.45,78.03,6.02
154,154,Yerbogachën,RU,2012956,0,41,61.28,108.01,79.48,7.45
174,174,Wuchang,CN,2034141,0,53,44.91,127.15,79.11,5.82
210,210,Zhangye,CN,1785036,0,26,38.93,100.45,78.03,6.02
229,229,Xining,CN,1788852,0,24,36.62,101.77,72.77,2.51
267,267,Vila velha,BR,3445026,0,83,-20.33,-40.29,72.00,3.36
314,314,Jinchang,CN,1805733,0,25,38.50,102.17,77.22,1.54
380,380,Poum,NC,2138555,0,62,-20.23,164.02,72.48,7.20
458,458,Mount isa,AU,2065594,0,27,-20.73,139.50,75.20,3.36
516,516,Lerici,IT,3174856,0,69,44.08,9.91,71.01,3.71


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Store into variable named hotel_df.
hotel_df = narrow_data

# Add a "Hotel Name" column to the DataFrame..
hotel_df["Hotel Name"] = np.nan

In [6]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    print(f"Querying for {row['City']}.")
    
    #Hit the Google Places API for each city's coordinates.
    params = {"type" : "hotel","keyword" : "hotel","radius" : 5000, "key" : g_key, "location" : str(row["Latitude"]) + "," + str(row["Longitude"])}
    response = requests.get(url, params=params).json()
    results = response['results']

    try:
        #Store the first Hotel result into the DataFrame.
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        time.sleep(1)
        pass
    
    time.sleep(1)
print("Finished queries")

Querying for Zhangye.
Querying for Yerbogachën.
Querying for Wuchang.
Querying for Zhangye.
Querying for Xining.
Querying for Vila velha.
Querying for Jinchang.
Querying for Poum.
Querying for Mount isa.
Querying for Lerici.
Querying for Kashary.
Querying for Mount isa.
Querying for Xining.
Querying for Yumen.
Querying for Vila velha.
Querying for Vila velha.
Querying for Yerbogachën.
Querying for Semnan.
Querying for Vila velha.
Querying for Fort-shevchenko.
Querying for Gilgit.
Finished queries


In [7]:
hotel_df

,Unnamed: 0,City,Country,ID,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
86,86,Zhangye,CN,1785036,0,26,38.93,100.45,78.03,6.02,Holiday Inn Express Zhangye
154,154,Yerbogachën,RU,2012956,0,41,61.28,108.01,79.48,7.45,NaN
174,174,Wuchang,CN,2034141,0,53,44.91,127.15,79.11,5.82,Vienna 3 Best Hotel
210,210,Zhangye,CN,1785036,0,26,38.93,100.45,78.03,6.02,Holiday Inn Express Zhangye
229,229,Xining,CN,1788852,0,24,36.62,101.77,72.77,2.51,Sofitel Xining
267,267,Vila velha,BR,3445026,0,83,-20.33,-40.29,72.00,3.36,Quality Suites Vila Velha
314,314,Jinchang,CN,1805733,0,25,38.50,102.17,77.22,1.54,Jinchang Hotel
380,380,Poum,NC,2138555,0,62,-20.23,164.02,72.48,7.20,NaN
458,458,Mount isa,AU,2065594,0,27,-20.73,139.50,75.20,3.36,Abacus Motel
516,516,Lerici,IT,3174856,0,69,44.08,9.91,71.01,3.71,Piccolo Hotel del Lido


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Plot markers on top of the heatmap.
fig.add_layer(gmaps.marker_layer(locations, info_box_content = hotel_info))
fig

Figure(layout=FigureLayout(height='420px'))